# 1. Imports

In [66]:
import pandas as pd
import numpy as np

from pathlib import Path
import pyarrow as pa

import matplotlib.pyplot as plt
from matplotlib.colors import to_hex
import seaborn as sns
from pylab import rcParams
import plotly.graph_objects as go

### Constants

In [2]:
SEED = 42

DATA_RAW_PATH = Path("../data/raw")
DATA_INTER_PATH = Path("../data/interim")
DATA_PROC_PATH = Path("../data/processed")
PIC_EXPLORATORY_PATH = Path("../reports/figures/exploratory")

TRAIN_DATA = "X_train.parquet"
TRAIN_DATA_Y = "y_train.parquet"
METAINFO = "messages.xlsx"
FLOAT_TYPE = "float32"
SAVE_EXT = ".csv"

FREQ = 10
NAN_VAL = -1
NEG_VAL = -1
NAN_MARK = 1
NEG_MARK = 2
MAX_MARK = 3
NEG_TRESH = 0

GROUP_FREQ = '1Min'
GROUP_FUNC = 'mean'
GROUP_FUNC_Y = 'median'

In [3]:
EQUIP_NAMES = ["ЭКСГАУСТЕР "+ str(i) for i in range(4, 10)]
FEATURE_NAMES= [
    'ТОК РОТОРА 1', 'ТОК РОТОРА 2', 'ТОК СТАТОРА', 
    'ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ',
    'ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1', 'ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2', 'ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3', 'ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4',
    'ТЕМПЕРАТУРА МАСЛА В СИСТЕМЕ', 'ТЕМПЕРАТУРА МАСЛА В МАСЛОБЛОКЕ',
    'ВИБРАЦИЯ НА ОПОРЕ 1', 'ВИБРАЦИЯ НА ОПОРЕ 2', 'ВИБРАЦИЯ НА ОПОРЕ 3', 'ВИБРАЦИЯ НА ОПОРЕ 4', 
    'ВИБРАЦИЯ НА ОПОРЕ 3. ПРОДОЛЬНАЯ.', 'ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.',
    ]
NAME_EXCLUSION = ('ЭКСГАУСТЕР 4. ТОК РОТОРА 2', 'ЭКСГАУСТЕР 4. ТОК РОТОРА2')
DATETIME_COL = "DT"

Plot params

In [4]:
# Matplotlib params
rcParams['figure.figsize'] = 24, 8
plt.set_loglevel('WARNING')

SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

rcParams['font.size'] = SMALL_SIZE          # controls default text sizes
rcParams['axes.titlesize'] = SMALL_SIZE     # fontsize of the axes title
rcParams['axes.labelsize'] = MEDIUM_SIZE    # fontsize of the x and y labels
rcParams['xtick.labelsize'] = SMALL_SIZE    # fontsize of the tick labels
rcParams['ytick.labelsize'] = SMALL_SIZE    # fontsize of the tick labels
rcParams['legend.fontsize'] = SMALL_SIZE    # legend fontsize
rcParams['figure.titlesize'] = BIGGER_SIZE  # fontsize of the figure title

sns.set_theme()
BASE_CONTEXT = 'fivethirtyeight'

# Plot
SHOW_ROWS = 600

# See palette
PALETTE = sns.color_palette()
print(PALETTE.as_hex())

['#4c72b0', '#dd8452', '#55a868', '#c44e52', '#8172b3', '#937860', '#da8bc3', '#8c8c8c', '#ccb974', '#64b5cd']


# 2. Data uploading

In [5]:
def df_describe_info(df: pd.DataFrame,
                     percentiles: list[float] =[.002, .01, .05, .5, .95, .99],
                     ) -> None:
    display(df.info(show_counts=True))
    display(df.describe(percentiles=percentiles))
    print(f"NaNs portion: {sum(df.isna().any(axis=1)) / len(df):.3f}")

## 2.1. Train x

In [6]:
df_train_cols = [eq_name + ". " + feature_name for eq_name in EQUIP_NAMES for feature_name in FEATURE_NAMES]
df_train_cols = [NAME_EXCLUSION[1] if item == NAME_EXCLUSION[0] else item for item in df_train_cols]

df_train_schema = pa.schema([(name, "float32") for name in df_train_cols] + [(DATETIME_COL, pa.timestamp('s'))])

### Common info

In [7]:
df_train = pd.read_parquet(DATA_RAW_PATH / TRAIN_DATA, schema=df_train_schema)
display(df_train.info(show_counts=True))
df_train.set_index(DATETIME_COL, drop=True, inplace=True)
df_train.rename({NAME_EXCLUSION[1]: NAME_EXCLUSION[0]}, axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9335034 entries, 0 to 9335033
Data columns (total 97 columns):
 #   Column                                           Non-Null Count    Dtype        
---  ------                                           --------------    -----        
 0   ЭКСГАУСТЕР 4. ТОК РОТОРА 1                       8502866 non-null  float32      
 1   ЭКСГАУСТЕР 4. ТОК РОТОРА2                        8496869 non-null  float32      
 2   ЭКСГАУСТЕР 4. ТОК СТАТОРА                        8864552 non-null  float32      
 3   ЭКСГАУСТЕР 4. ДАВЛЕНИЕ МАСЛА В СИСТЕМЕ           8635520 non-null  float32      
 4   ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 1  6425379 non-null  float32      
 5   ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 2  6322328 non-null  float32      
 6   ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 3  6365231 non-null  float32      
 7   ЭКСГАУСТЕР 4. ТЕМПЕРАТУРА ПОДШИПНИКА НА ОПОРЕ 4  6417749 non-null  float32      
 8   ЭКСГАУСТЕР 4. ТЕМПЕРАТ

None

### Separate info by eqiupment

In [8]:
df_train_dict = {}
for equip_name in EQUIP_NAMES:
    df_train_dict[equip_name.lower()] = df_train[df_train.columns[df_train.columns.str.startswith(equip_name)]]
    df_train_dict[equip_name.lower()].columns = df_train_dict[equip_name.lower()].columns.str[len(EQUIP_NAMES[0] + ". "):].str.lower()
    df_train_dict[equip_name.lower()].index.name = df_train_dict[equip_name.lower()].index.name.lower()

Lowercase column names etc.

In [9]:
EQUIP_NAMES = [equip_name.lower() for equip_name in EQUIP_NAMES]
FEATURE_NAMES = [feature_name.lower() for feature_name in FEATURE_NAMES]
DATETIME_COL = DATETIME_COL.lower()

Dataset timerange.

In [10]:
df_train_start_end = (df_train_dict[EQUIP_NAMES[0]].index.min(), df_train_dict[EQUIP_NAMES[0]].index.max())
df_train_shape = df_train_dict[EQUIP_NAMES[0]].shape
print("Date period: ", df_train_start_end)
print("Shape: ", df_train_shape)

Date period:  (Timestamp('2019-01-16 13:21:00'), Timestamp('2021-12-31 23:59:50'))
Shape:  (9335034, 16)


### Separate info by eqiupment

In [11]:
test_eqip = EQUIP_NAMES[0]

Time frequency

In [12]:
dt_diff = (df_train_dict[test_eqip].index[1:] - df_train_dict[test_eqip].index[:-1]).seconds
print("Mean, min, max of datetime column:", np.mean(dt_diff), np.min(dt_diff), np.max(dt_diff))

Mean, min, max of datetime column: 10.0 10 10


In [13]:
# for equip_name in EQUIP_NAMES:
#     df_train_dict[equip_name] = df_train_dict[equip_name].dropna(how="all")

### Analyse one of the equipment

In [14]:
df_describe_info(df_train_dict[test_eqip])

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9335034 entries, 2019-01-16 13:21:00 to 2021-12-31 23:59:50
Data columns (total 16 columns):
 #   Column                             Non-Null Count    Dtype  
---  ------                             --------------    -----  
 0   ток ротора 1                       8502866 non-null  float32
 1   ток ротора 2                       8496869 non-null  float32
 2   ток статора                        8864552 non-null  float32
 3   давление масла в системе           8635520 non-null  float32
 4   температура подшипника на опоре 1  6425379 non-null  float32
 5   температура подшипника на опоре 2  6322328 non-null  float32
 6   температура подшипника на опоре 3  6365231 non-null  float32
 7   температура подшипника на опоре 4  6417749 non-null  float32
 8   температура масла в системе        6375755 non-null  float32
 9   температура масла в маслоблоке     6322877 non-null  float32
 10  вибрация на опоре 1                8532819 non-null  floa

None

,ток ротора 1,ток ротора 2,ток статора,давление масла в системе,температура подшипника на опоре 1,температура подшипника на опоре 2,температура подшипника на опоре 3,температура подшипника на опоре 4,температура масла в системе,температура масла в маслоблоке,вибрация на опоре 1,вибрация на опоре 2,вибрация на опоре 3,вибрация на опоре 4,вибрация на опоре 3. продольная.,вибрация на опоре 4. продольная.
count,8.502866e+06,8.496869e+06,8.864552e+06,8.635520e+06,6.425379e+06,6.322328e+06,6.365231e+06,6.417749e+06,6.375755e+06,6.322877e+06,8.532819e+06,8.407650e+06,8.519049e+06,8.915882e+06,8.857177e+06,8.498674e+06
mean,2.856738e+02,2.859001e+02,2.571010e+02,1.310709e+02,4.745253e+01,4.616409e+01,4.263668e+01,4.491410e+01,3.632390e+01,4.200555e+01,1.273967e+00,9.407912e-01,1.179088e+00,1.591222e+00,3.105385e+00,1.229493e+00
std,6.387723e+01,6.339924e+01,4.500917e+01,2.662698e+01,7.173366e+00,7.068939e+00,6.088381e+00,6.002651e+00,8.192476e+00,5.553409e+00,4.843087e-01,3.745172e-01,8.630037e-01,3.100204e+00,1.865003e+00,7.942830e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+02,-1.000000e+02,0.000000e+00,-1.620000e+00,-8.100000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+02,0.000000e+00,0.000000e+00
0.2%,6.076389e-01,6.076389e-01,1.736111e-01,0.000000e+00,6.490133e+00,1.055000e+01,8.112663e+00,8.842802e+00,8.112663e+00,1.217000e+01,2.777778e-02,6.000000e-02,6.000000e-02,3.000000e-02,8.000000e-02,7.000000e-02
1%,1.502893e+00,1.502893e+00,3.559028e-01,1.543210e-01,1.379000e+01,1.785000e+01,1.623000e+01,1.776675e+01,1.136000e+01,2.109294e+01,5.381944e-02,9.000000e-02,8.333333e-02,6.000000e-02,9.452160e-02,1.100000e-01
5%,2.278125e+02,2.278220e+02,2.415800e+02,1.008100e+02,3.651000e+01,3.695771e+01,3.326000e+01,3.651000e+01,1.793802e+01,3.326000e+01,3.900000e-01,4.300000e-01,2.400000e-01,6.200000e-01,8.940972e-01,3.300000e-01
50%,2.740419e+02,2.742780e+02,2.618400e+02,1.292800e+02,4.867601e+01,4.624000e+01,4.344785e+01,4.551207e+01,3.812955e+01,4.234812e+01,1.360000e+00,8.619791e-01,8.603395e-01,1.460000e+00,2.560000e+00,1.040000e+00
95%,3.812457e+02,3.812703e+02,2.837480e+02,1.702780e+02,5.579713e+01,5.678868e+01,5.029855e+01,5.210136e+01,4.672896e+01,4.964953e+01,2.013889e+00,1.670000e+00,2.780000e+00,3.488619e+00,6.790000e+00,2.800000e+00
99%,3.983150e+02,3.983150e+02,2.929840e+02,1.855300e+02,5.814079e+01,6.076390e+01,5.321910e+01,5.480559e+01,4.981179e+01,5.273234e+01,2.220000e+00,1.921875e+00,3.666088e+00,4.235243e+00,8.120000e+00,3.289062e+00


NaNs portion: 0.463


In [15]:
display(df_train_dict[test_eqip].info(show_counts=True))
display(df_train_dict[test_eqip].describe(percentiles=[.002, .01, .05, .5, .95, .99]))
print(f"NaNs portion: {sum(df_train_dict[test_eqip].isna().any(axis=1)) / len(df_train_dict[test_eqip]):.3f}")

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9335034 entries, 2019-01-16 13:21:00 to 2021-12-31 23:59:50
Data columns (total 16 columns):
 #   Column                             Non-Null Count    Dtype  
---  ------                             --------------    -----  
 0   ток ротора 1                       8502866 non-null  float32
 1   ток ротора 2                       8496869 non-null  float32
 2   ток статора                        8864552 non-null  float32
 3   давление масла в системе           8635520 non-null  float32
 4   температура подшипника на опоре 1  6425379 non-null  float32
 5   температура подшипника на опоре 2  6322328 non-null  float32
 6   температура подшипника на опоре 3  6365231 non-null  float32
 7   температура подшипника на опоре 4  6417749 non-null  float32
 8   температура масла в системе        6375755 non-null  float32
 9   температура масла в маслоблоке     6322877 non-null  float32
 10  вибрация на опоре 1                8532819 non-null  floa

None

,ток ротора 1,ток ротора 2,ток статора,давление масла в системе,температура подшипника на опоре 1,температура подшипника на опоре 2,температура подшипника на опоре 3,температура подшипника на опоре 4,температура масла в системе,температура масла в маслоблоке,вибрация на опоре 1,вибрация на опоре 2,вибрация на опоре 3,вибрация на опоре 4,вибрация на опоре 3. продольная.,вибрация на опоре 4. продольная.
count,8.502866e+06,8.496869e+06,8.864552e+06,8.635520e+06,6.425379e+06,6.322328e+06,6.365231e+06,6.417749e+06,6.375755e+06,6.322877e+06,8.532819e+06,8.407650e+06,8.519049e+06,8.915882e+06,8.857177e+06,8.498674e+06
mean,2.856738e+02,2.859001e+02,2.571010e+02,1.310709e+02,4.745253e+01,4.616409e+01,4.263668e+01,4.491410e+01,3.632390e+01,4.200555e+01,1.273967e+00,9.407912e-01,1.179088e+00,1.591222e+00,3.105385e+00,1.229493e+00
std,6.387723e+01,6.339924e+01,4.500917e+01,2.662698e+01,7.173366e+00,7.068939e+00,6.088381e+00,6.002651e+00,8.192476e+00,5.553409e+00,4.843087e-01,3.745172e-01,8.630037e-01,3.100204e+00,1.865003e+00,7.942830e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+02,-1.000000e+02,0.000000e+00,-1.620000e+00,-8.100000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+02,0.000000e+00,0.000000e+00
0.2%,6.076389e-01,6.076389e-01,1.736111e-01,0.000000e+00,6.490133e+00,1.055000e+01,8.112663e+00,8.842802e+00,8.112663e+00,1.217000e+01,2.777778e-02,6.000000e-02,6.000000e-02,3.000000e-02,8.000000e-02,7.000000e-02
1%,1.502893e+00,1.502893e+00,3.559028e-01,1.543210e-01,1.379000e+01,1.785000e+01,1.623000e+01,1.776675e+01,1.136000e+01,2.109294e+01,5.381944e-02,9.000000e-02,8.333333e-02,6.000000e-02,9.452160e-02,1.100000e-01
5%,2.278125e+02,2.278220e+02,2.415800e+02,1.008100e+02,3.651000e+01,3.695771e+01,3.326000e+01,3.651000e+01,1.793802e+01,3.326000e+01,3.900000e-01,4.300000e-01,2.400000e-01,6.200000e-01,8.940972e-01,3.300000e-01
50%,2.740419e+02,2.742780e+02,2.618400e+02,1.292800e+02,4.867601e+01,4.624000e+01,4.344785e+01,4.551207e+01,3.812955e+01,4.234812e+01,1.360000e+00,8.619791e-01,8.603395e-01,1.460000e+00,2.560000e+00,1.040000e+00
95%,3.812457e+02,3.812703e+02,2.837480e+02,1.702780e+02,5.579713e+01,5.678868e+01,5.029855e+01,5.210136e+01,4.672896e+01,4.964953e+01,2.013889e+00,1.670000e+00,2.780000e+00,3.488619e+00,6.790000e+00,2.800000e+00
99%,3.983150e+02,3.983150e+02,2.929840e+02,1.855300e+02,5.814079e+01,6.076390e+01,5.321910e+01,5.480559e+01,4.981179e+01,5.273234e+01,2.220000e+00,1.921875e+00,3.666088e+00,4.235243e+00,8.120000e+00,3.289062e+00


NaNs portion: 0.463


* Many NaNs - 0,463. Some parameters have more null, e.g. all temperatures
* Also "ток статора" имеет очень высокие значения (600А), "давление масла в системе" (4 избыт. атм.?), "температура подшипника" 486 C, вибрации на опорах (на опорах 1 и 2 - выше 2, на 3 и 4 - 3,5; продольные - 3-8)
* Negative values looks anomalies

NaNs in one row

In [16]:
print("NaNs in one row:")
for col in df_train_dict[test_eqip].columns:
    nans_count = df_train_dict[test_eqip][col].isnull().astype(int).groupby(df_train_dict[test_eqip][col].notnull().astype(int).cumsum()).sum()
    print(f"Column {col} in hours {nans_count.max() * FREQ / 60 / 60:.1f} hours and {nans_count.max()} counts")

NaNs in one row:
Column ток ротора 1 in hours 232.3 hours and 83630 counts
Column ток ротора 2 in hours 232.3 hours and 83630 counts
Column ток статора in hours 232.3 hours and 83630 counts
Column давление масла в системе in hours 232.3 hours and 83630 counts
Column температура подшипника на опоре 1 in hours 232.3 hours and 83630 counts
Column температура подшипника на опоре 2 in hours 232.3 hours and 83630 counts
Column температура подшипника на опоре 3 in hours 232.3 hours and 83630 counts
Column температура подшипника на опоре 4 in hours 232.3 hours and 83631 counts
Column температура масла в системе in hours 232.3 hours and 83637 counts
Column температура масла в маслоблоке in hours 232.3 hours and 83636 counts
Column вибрация на опоре 1 in hours 232.3 hours and 83630 counts
Column вибрация на опоре 2 in hours 232.3 hours and 83630 counts
Column вибрация на опоре 3 in hours 232.3 hours and 83630 counts
Column вибрация на опоре 4 in hours 232.3 hours and 83630 counts
Column вибрация

One of the periond have NaNs during 232 hours. Without such case (when all NaNs) NaNs in row ~ 30 hours.

### Aggreagte data

In [17]:
for equip_name in EQUIP_NAMES:
    df_train_dict[equip_name] = df_train_dict[equip_name].groupby(pd.Grouper(freq=GROUP_FREQ)).aggregate(GROUP_FUNC)

Was 9335034, now 1555839 rows

### Save NaN's positions and doubtful values

* `df_train_state_dict` - store indicators of bad values or __NaNs__
* for `df_train_dict` - replace __NaNs__ and __negative values__

In [18]:
df_train_state_dict = {equip_name: pd.DataFrame(0, index=df_train_dict[equip_name].index, columns=df_train_dict[equip_name].columns, dtype=np.int8) for equip_name in EQUIP_NAMES}

for equip_name in EQUIP_NAMES:
    df_train_state_dict[equip_name][df_train_dict[equip_name].isnull()] = NAN_MARK
    df_train_state_dict[equip_name][df_train_dict[equip_name] < NEG_TRESH] = NEG_MARK
    df_train_dict[equip_name][df_train_dict[equip_name].isnull()] = NAN_VAL
    df_train_dict[equip_name][df_train_dict[equip_name] < NEG_TRESH] = NEG_VAL

In [19]:
df_describe_info(df_train_dict[test_eqip])

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1555839 entries, 2019-01-16 13:21:00 to 2021-12-31 23:59:00
Freq: T
Data columns (total 16 columns):
 #   Column                             Non-Null Count    Dtype  
---  ------                             --------------    -----  
 0   ток ротора 1                       1555839 non-null  float32
 1   ток ротора 2                       1555839 non-null  float32
 2   ток статора                        1555839 non-null  float32
 3   давление масла в системе           1555839 non-null  float32
 4   температура подшипника на опоре 1  1555839 non-null  float32
 5   температура подшипника на опоре 2  1555839 non-null  float32
 6   температура подшипника на опоре 3  1555839 non-null  float32
 7   температура подшипника на опоре 4  1555839 non-null  float32
 8   температура масла в системе        1555839 non-null  float32
 9   температура масла в маслоблоке     1555839 non-null  float32
 10  вибрация на опоре 1                1555839 non-nu

None

,ток ротора 1,ток ротора 2,ток статора,давление масла в системе,температура подшипника на опоре 1,температура подшипника на опоре 2,температура подшипника на опоре 3,температура подшипника на опоре 4,температура масла в системе,температура масла в маслоблоке,вибрация на опоре 1,вибрация на опоре 2,вибрация на опоре 3,вибрация на опоре 4,вибрация на опоре 3. продольная.,вибрация на опоре 4. продольная.
count,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06
mean,2.718409e+02,2.718725e+02,2.458754e+02,1.259223e+02,4.033101e+01,3.905657e+01,3.558645e+01,3.798860e+01,3.071867e+01,3.500560e+01,1.190418e+00,8.777451e-01,1.113310e+00,1.596157e+00,2.995057e+00,1.158709e+00
std,8.388812e+01,8.384470e+01,6.798654e+01,3.657796e+01,1.767732e+01,1.783024e+01,1.615249e+01,1.632980e+01,1.475874e+01,1.597877e+01,6.087348e-01,4.851554e-01,9.194074e-01,9.636254e-01,1.955154e+00,8.631763e-01
min,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
0.2%,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
1%,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
5%,1.714445e+00,1.508727e+00,2.604167e-01,6.963735e-01,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,1.075000e-01,1.360000e-01,1.125000e-01,1.000000e-01,1.216667e-01,1.850000e-01
50%,2.524988e+02,2.525596e+02,2.611915e+02,1.280792e+02,4.664500e+01,4.502500e+01,4.162172e+01,4.379338e+01,3.563001e+01,4.084595e+01,1.336667e+00,8.433333e-01,8.383333e-01,1.428418e+00,2.501667e+00,9.969457e-01
95%,3.791037e+02,3.791037e+02,2.834238e+02,1.698699e+02,5.517967e+01,5.570556e+01,4.955489e+01,5.151500e+01,4.596925e+01,4.899301e+01,1.998587e+00,1.655000e+00,2.753940e+00,3.386936e+00,6.760000e+00,2.767819e+00
99%,3.979829e+02,3.979830e+02,2.927658e+02,1.852080e+02,5.786679e+01,5.949000e+01,5.279994e+01,5.435489e+01,4.926043e+01,5.232500e+01,2.212050e+00,1.906473e+00,3.644895e+00,4.034119e+00,8.088997e+00,3.267162e+00


NaNs portion: 0.000


In [20]:
display(df_train_dict[test_eqip].info(show_counts=True))
display(df_train_dict[test_eqip].describe(percentiles=[.002, .01, .05, .5, .95, .99]))
print(f"NaNs portion: {sum(df_train_dict[test_eqip].isna().any(axis=1)) / len(df_train_dict[test_eqip]):.3f}")

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1555839 entries, 2019-01-16 13:21:00 to 2021-12-31 23:59:00
Freq: T
Data columns (total 16 columns):
 #   Column                             Non-Null Count    Dtype  
---  ------                             --------------    -----  
 0   ток ротора 1                       1555839 non-null  float32
 1   ток ротора 2                       1555839 non-null  float32
 2   ток статора                        1555839 non-null  float32
 3   давление масла в системе           1555839 non-null  float32
 4   температура подшипника на опоре 1  1555839 non-null  float32
 5   температура подшипника на опоре 2  1555839 non-null  float32
 6   температура подшипника на опоре 3  1555839 non-null  float32
 7   температура подшипника на опоре 4  1555839 non-null  float32
 8   температура масла в системе        1555839 non-null  float32
 9   температура масла в маслоблоке     1555839 non-null  float32
 10  вибрация на опоре 1                1555839 non-nu

None

,ток ротора 1,ток ротора 2,ток статора,давление масла в системе,температура подшипника на опоре 1,температура подшипника на опоре 2,температура подшипника на опоре 3,температура подшипника на опоре 4,температура масла в системе,температура масла в маслоблоке,вибрация на опоре 1,вибрация на опоре 2,вибрация на опоре 3,вибрация на опоре 4,вибрация на опоре 3. продольная.,вибрация на опоре 4. продольная.
count,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06
mean,2.718409e+02,2.718725e+02,2.458754e+02,1.259223e+02,4.033101e+01,3.905657e+01,3.558645e+01,3.798860e+01,3.071867e+01,3.500560e+01,1.190418e+00,8.777451e-01,1.113310e+00,1.596157e+00,2.995057e+00,1.158709e+00
std,8.388812e+01,8.384470e+01,6.798654e+01,3.657796e+01,1.767732e+01,1.783024e+01,1.615249e+01,1.632980e+01,1.475874e+01,1.597877e+01,6.087348e-01,4.851554e-01,9.194074e-01,9.636254e-01,1.955154e+00,8.631763e-01
min,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
0.2%,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
1%,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
5%,1.714445e+00,1.508727e+00,2.604167e-01,6.963735e-01,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,1.075000e-01,1.360000e-01,1.125000e-01,1.000000e-01,1.216667e-01,1.850000e-01
50%,2.524988e+02,2.525596e+02,2.611915e+02,1.280792e+02,4.664500e+01,4.502500e+01,4.162172e+01,4.379338e+01,3.563001e+01,4.084595e+01,1.336667e+00,8.433333e-01,8.383333e-01,1.428418e+00,2.501667e+00,9.969457e-01
95%,3.791037e+02,3.791037e+02,2.834238e+02,1.698699e+02,5.517967e+01,5.570556e+01,4.955489e+01,5.151500e+01,4.596925e+01,4.899301e+01,1.998587e+00,1.655000e+00,2.753940e+00,3.386936e+00,6.760000e+00,2.767819e+00
99%,3.979829e+02,3.979830e+02,2.927658e+02,1.852080e+02,5.786679e+01,5.949000e+01,5.279994e+01,5.435489e+01,4.926043e+01,5.232500e+01,2.212050e+00,1.906473e+00,3.644895e+00,4.034119e+00,8.088997e+00,3.267162e+00


NaNs portion: 0.000


In [21]:
df_describe_info(df_train_state_dict[test_eqip])

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1555839 entries, 2019-01-16 13:21:00 to 2021-12-31 23:59:00
Freq: T
Data columns (total 16 columns):
 #   Column                             Non-Null Count    Dtype
---  ------                             --------------    -----
 0   ток ротора 1                       1555839 non-null  int8 
 1   ток ротора 2                       1555839 non-null  int8 
 2   ток статора                        1555839 non-null  int8 
 3   давление масла в системе           1555839 non-null  int8 
 4   температура подшипника на опоре 1  1555839 non-null  int8 
 5   температура подшипника на опоре 2  1555839 non-null  int8 
 6   температура подшипника на опоре 3  1555839 non-null  int8 
 7   температура подшипника на опоре 4  1555839 non-null  int8 
 8   температура масла в системе        1555839 non-null  int8 
 9   температура масла в маслоблоке     1555839 non-null  int8 
 10  вибрация на опоре 1                1555839 non-null  int8 
 11  вибрация 

None

,ток ротора 1,ток ротора 2,ток статора,давление масла в системе,температура подшипника на опоре 1,температура подшипника на опоре 2,температура подшипника на опоре 3,температура подшипника на опоре 4,температура масла в системе,температура масла в маслоблоке,вибрация на опоре 1,вибрация на опоре 2,вибрация на опоре 3,вибрация на опоре 4,вибрация на опоре 3. продольная.,вибрация на опоре 4. продольная.
count,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06
mean,3.859397e-02,3.919943e-02,4.166755e-02,3.949959e-02,1.341900e-01,1.475770e-01,1.446339e-01,1.327528e-01,1.385754e-01,1.488265e-01,2.725925e-02,2.721297e-02,2.761725e-02,2.848688e-02,2.391957e-02,2.739808e-02
std,1.926253e-01,1.940692e-01,1.998284e-01,1.947804e-01,3.409809e-01,3.552470e-01,3.517314e-01,3.394249e-01,3.456834e-01,3.559174e-01,1.628379e-01,1.627035e-01,1.638736e-01,1.711731e-01,1.527987e-01,1.632404e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
0.2%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
5%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
95%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
99%,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


NaNs portion: 0.000


In [22]:
display(df_train_state_dict[test_eqip].info(show_counts=True))
display(df_train_state_dict[test_eqip].describe(percentiles=[.002, .01, .05, .5, .95, .99]))
print(f"NaNs portion: {sum(df_train_state_dict[test_eqip].isna().any(axis=1)) / len(df_train_state_dict[test_eqip]):.3f}")

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1555839 entries, 2019-01-16 13:21:00 to 2021-12-31 23:59:00
Freq: T
Data columns (total 16 columns):
 #   Column                             Non-Null Count    Dtype
---  ------                             --------------    -----
 0   ток ротора 1                       1555839 non-null  int8 
 1   ток ротора 2                       1555839 non-null  int8 
 2   ток статора                        1555839 non-null  int8 
 3   давление масла в системе           1555839 non-null  int8 
 4   температура подшипника на опоре 1  1555839 non-null  int8 
 5   температура подшипника на опоре 2  1555839 non-null  int8 
 6   температура подшипника на опоре 3  1555839 non-null  int8 
 7   температура подшипника на опоре 4  1555839 non-null  int8 
 8   температура масла в системе        1555839 non-null  int8 
 9   температура масла в маслоблоке     1555839 non-null  int8 
 10  вибрация на опоре 1                1555839 non-null  int8 
 11  вибрация 

None

,ток ротора 1,ток ротора 2,ток статора,давление масла в системе,температура подшипника на опоре 1,температура подшипника на опоре 2,температура подшипника на опоре 3,температура подшипника на опоре 4,температура масла в системе,температура масла в маслоблоке,вибрация на опоре 1,вибрация на опоре 2,вибрация на опоре 3,вибрация на опоре 4,вибрация на опоре 3. продольная.,вибрация на опоре 4. продольная.
count,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06,1.555839e+06
mean,3.859397e-02,3.919943e-02,4.166755e-02,3.949959e-02,1.341900e-01,1.475770e-01,1.446339e-01,1.327528e-01,1.385754e-01,1.488265e-01,2.725925e-02,2.721297e-02,2.761725e-02,2.848688e-02,2.391957e-02,2.739808e-02
std,1.926253e-01,1.940692e-01,1.998284e-01,1.947804e-01,3.409809e-01,3.552470e-01,3.517314e-01,3.394249e-01,3.456834e-01,3.559174e-01,1.628379e-01,1.627035e-01,1.638736e-01,1.711731e-01,1.527987e-01,1.632404e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
0.2%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
5%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
95%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
99%,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


NaNs portion: 0.000


### Save data

In [23]:
# for equip_name in EQUIP_NAMES:
#     df_train_dict[equip_name].to_csv(DATA_INTER_PATH / (equip_name + "_train" + SAVE_EXT))
# for equip_name in EQUIP_NAMES:
#     df_train_state_dict[equip_name].to_csv(DATA_INTER_PATH / (equip_name + "_train_idx" + SAVE_EXT))

## 2.2. Train y

In [24]:
Y_NAMES = [
    'Y_ЭКСГАУСТЕР А/М №9_ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №9',
    'Y_ЭКСГАУСТЕР А/М №9_МАСЛОСТАНЦИЯ ЖИДКОЙ СМАЗКИ ЭКСГ. №9',
    'Y_ЭКСГАУСТЕР А/М №9_ЭЛЕКТРОАППАРАТУРА ЭКСГ. №9',
    'Y_ЭКСГАУСТЕР А/М №9_КЛ1 ТР№9 ДО ЭД ЭКСГАУСТЕРА №9',
    'Y_ЭКСГАУСТЕР А/М №9_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №9',
    'Y_ЭКСГАУСТЕР А/М №9_КОРПУС ЭКСГ. №9',
    'Y_ЭКСГАУСТЕР А/М №9_РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №9',
    'Y_ЭКСГАУСТЕР А/М №9_МАСЛООХЛАДИТЕЛЬ  М-05-1 ЭКСГ. №9',
    'Y_ЭКСГАУСТЕР А/М №9_ЭЛ/ДВИГАТЕЛЬ ГАЗ. ЗАДВИЖКИ ЭКСГ. №9',
    'Y_ЭКСГАУСТЕР А/М №9_ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №9',
    'Y_ЭКСГАУСТЕР А/М №9_ПОДШИПНИК ОПОРНО-УПОРНЫЙ ЭКСГ. №9',
    'Y_ЭКСГАУСТЕР А/М №9_РОТОР ЭКСГ. №9',
    'Y_ЭКСГАУСТЕР А/М №9_ТР-Р ТМ-6300-10/6 ЭКСГ. №9',
    'Y_ЭКСГАУСТЕР А/М №9_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №9',
    'Y_ЭКСГАУСТЕР А/М №9_МАСЛОНАСОС РАБОЧИЙ ЭКСГ. №9',
    'Y_ЭКСГАУСТЕР А/М №9_ПОДШИПНИК ОПОРНЫЙ №2',
    'Y_ЭКСГАУСТЕР А/М №9_ГСМ ЭКСГ. №9',
    'Y_ЭКСГАУСТЕР А/М №9_КОЖУХ МУФТЫ ЭКСГ. №9',
    'Y_ЭКСГАУСТЕР А/М №9_ПОДШИПНИК ОПОРНЫЙ №1',
    'Y_ЭКСГАУСТЕР А/М №9_ЗАДВИЖКА ЭКСГ. №9',
    'Y_ЭКСГАУСТЕР А/М №9_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№9 Т.1',
    'Y_ЭКСГАУСТЕР А/М №9_МАСЛОПРОВОДЫ ЭКСГАУСТЕРА №9',
    'Y_ЭКСГАУСТЕР А/М №9_ТИРИСТОРНЫЙ ВОЗБУДИТЕЛЬ ТВ-400 ЭКСГ9 ВУ1',
    'Y_ЭКСГАУСТЕР А/М №9_МЕТРАН-100 ДАТЧИКИ ДАВЛЕНИЯ ЭКСГ.№9',
    'Y_ЭКСГАУСТЕР А/М №9_ТСМТ-101-010-50М-80 ТЕРМОПРЕОБР.МАСЛО',
    'Y_ЭКСГАУСТЕР А/М №9_ТСМТ-101-010-50М-400 ТЕРМОПР.ПОДШ.Т.1',
    'Y_ЭКСГАУСТЕР А/М №9_ЭКСГАУСТЕР А/М №9',
    'Y_ЭКСГАУСТЕР А/М №9_ДВИГАТЕЛЬ ПУСКОВОГО МАСЛОНАСОСА ЭКСГ. №9',
    'Y_ЭКСГАУСТЕР А/М №5_МАСЛООХЛАДИТЕЛЬ  М-05-1 ЭКСГ. №5',
    'Y_ЭКСГАУСТЕР А/М №5_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №5',
    'Y_ЭКСГАУСТЕР А/М №5_КОРПУС ЭКСГ. №5',
    'Y_ЭКСГАУСТЕР А/М №5_ЗАДВИЖКА ЭКСГ. №5',
    'Y_ЭКСГАУСТЕР А/М №5_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №5',
    'Y_ЭКСГАУСТЕР А/М №5_ПОДШИПНИК ОПОРНЫЙ №2',
    'Y_ЭКСГАУСТЕР А/М №5_ПОДШИПНИК ОПОРНО-УПОРНЫЙ ЭКСГ. №5',
    'Y_ЭКСГАУСТЕР А/М №5_МАСЛОНАСОС ШЕСТЕРЕНЧАТЫЙ (РЕЗ.) ЭКСГ. №5',
    'Y_ЭКСГАУСТЕР А/М №5_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№5 Т.1',
    'Y_ЭКСГАУСТЕР А/М №5_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№5 Т.3',
    'Y_ЭКСГАУСТЕР А/М №5_ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №5',
    'Y_ЭКСГАУСТЕР А/М №5_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№5 Т.2',
    'Y_ЭКСГАУСТЕР А/М №5_ПОДШИПНИК ОПОРНЫЙ №1',
    'Y_ЭКСГАУСТЕР А/М №5_РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №5',
    'Y_ЭКСГАУСТЕР А/М №5_ТИРИСТ. ВОЗБУДИТЕЛЬ ВТ-РЭМ-400 ЭКСГ5 ВУ1',
    'Y_ЭКСГАУСТЕР А/М №5_ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №5',
    'Y_ЭКСГАУСТЕР А/М №5_МАСЛОПРОВОДЫ ЭКСГАУСТЕРА №5',
    'Y_ЭКСГАУСТЕР А/М №5_ГСМ ЭКСГ. №5',
    'Y_ЭКСГАУСТЕР А/М №5_ЭКСГАУСТЕР А/М №5',
    'Y_ЭКСГАУСТЕР А/М №5_ТСМТ-101-010-50М-80 ТЕРМОПРЕОБР.МАСЛО',
    'Y_ЭКСГАУСТЕР А/М №5_ТР-Р ТМ-4000-10/6 ЭКСГ. №5',
    'Y_ЭКСГАУСТЕР А/М №5_КЛ1 ТР№5 ДО ЭД ЭКСГАУСТЕРА №5',
    'Y_ЭКСГАУСТЕР А/М №5_КЛ2 ТР№5 ДО ЭД ЭКСГАУСТЕРА №5',
    'Y_ЭКСГАУСТЕР А/М №5_МАСЛОНАСОС РАБОЧИЙ ЭКСГ. №5',
    'Y_ЭКСГАУСТЕР А/М №5_РОТОР ЭКСГ. №5',
    'Y_ЭКСГАУСТЕР А/М №5_МЕТРАН-100 ДАТЧИКИ ДАВЛЕНИЯ ЭКСГ.№5',
    'Y_ЭКСГАУСТЕР А/М №5_ДВИГАТЕЛЬ ПУСКОВОГО МАСЛОНАСОСА ЭКСГ. №5',
    'Y_ЭКСГАУСТЕР А/М №5_ГАЗОВАЯ ЗАДВИЖКА ЭКСГАУСТЕРА А/М №5',
    'Y_ЭКСГАУСТЕР А/М №5_КОЖУХ МУФТЫ ЭКСГ. №5',
    'Y_ЭКСГАУСТЕР А/М №5_ТИРИСТ. ВОЗБУДИТЕЛЬ ВТ-РЭМ-400 ЭКСГ5 ВУ2',
    'Y_ЭКСГАУСТЕР А/М №5_ТСМТ-101-010-50М-400 ТЕРМОПР.ПОДШ.Т.2',
    'Y_ЭКСГАУСТЕР А/М №5_ДВИГАТЕЛЬ РЕЗЕРВНОГО МАСЛОНАСОСА ЭКСГ.№5',
    'Y_ЭКСГАУСТЕР А/М №5_ТСМТ-101-010-50М-200 ТЕРМОПР.ПОДШ.Т.3',
    'Y_ЭКСГАУСТЕР А/М №5_ЭКСГАУСТЕР Н-8000 А/М №5',
    'Y_ЭКСГАУСТЕР А/М №8_РОТОР ЭКСГ. №8',
    'Y_ЭКСГАУСТЕР А/М №8_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №8',
    'Y_ЭКСГАУСТЕР А/М №8_КОРПУС ЭКСГ. №8',
    'Y_ЭКСГАУСТЕР А/М №8_РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №8',
    'Y_ЭКСГАУСТЕР А/М №8_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№8 Т.3',
    'Y_ЭКСГАУСТЕР А/М №8_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№8 Т.4',
    'Y_ЭКСГАУСТЕР А/М №8_ЭЛ/ДВИГАТЕЛЬ ГАЗ. ЗАДВИЖКИ ЭКСГ. №8',
    'Y_ЭКСГАУСТЕР А/М №8_МАСЛООХЛАДИТЕЛЬ  М-05-1 ЭКСГ. №8',
    'Y_ЭКСГАУСТЕР А/М №8_ТИРИСТОРНЫЙ ВОЗБУДИТЕЛЬ СПВД-М10-400-5',
    'Y_ЭКСГАУСТЕР А/М №8_КЛ2 ТР№8 ДО ЭД ЭКСГАУСТЕРА №8',
    'Y_ЭКСГАУСТЕР А/М №8_КЛ1 ТР№8 ДО ЭД ЭКСГАУСТЕРА №8',
    'Y_ЭКСГАУСТЕР А/М №8_ТСМТ-101-010-50М-400 ТЕРМОПР.ПОДШ.Т.2',
    'Y_ЭКСГАУСТЕР А/М №8_КОЖУХ МУФТЫ ЭКСГ. №8',
    'Y_ЭКСГАУСТЕР А/М №8_ГСМ ЭКСГ. №8',
    'Y_ЭКСГАУСТЕР А/М №8_ПОДШИПНИК ОПОРНЫЙ №2',
    'Y_ЭКСГАУСТЕР А/М №8_ЭКСГАУСТЕР А/М №8',
    'Y_ЭКСГАУСТЕР А/М №8_ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №8',
    'Y_ЭКСГАУСТЕР А/М №8_ТСМТ-101-010-50М-200 ТЕРМОПР.ПОДШ.Т.4',
    'Y_ЭКСГАУСТЕР А/М №8_МАСЛЯНЫЙ ФИЛЬТР ЭКСГАУСТЕРА №8',
    'Y_ЭКСГАУСТЕР А/М №8_ТИРИСТОРНЫЙ ВОЗБУДИТЕЛЬ ТВ-400 ЭКСГ8 ВУ1',
    'Y_ЭКСГАУСТЕР А/М №8_ПОДШИПНИК ОПОРНЫЙ №1',
    'Y_ЭКСГАУСТЕР А/М №8_ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №8',
    'Y_ЭКСГАУСТЕР А/М №8_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №8',
    'Y_ЭКСГАУСТЕР А/М №8_ЗАДВИЖКА ЭКСГ. №8',
    'Y_ЭКСГАУСТЕР А/М №8_МАСЛОНАСОС РАБОЧИЙ ЭКСГ. №8',
    'Y_ЭКСГАУСТЕР А/М №8_МАСЛОНАСОС ШЕСТЕРЕНЧАТЫЙ (РЕЗ.) ЭКСГ. №8',
    'Y_ЭКСГАУСТЕР А/М №8_МАСЛОНАСОС ШЕСТЕРЕНЧАТЫЙ (ПУСК.) ЭКСГ.№8',
    'Y_ЭКСГАУСТЕР А/М №8_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№8 Т.2',
    'Y_ЭКСГАУСТЕР А/М №8_ТР-Р ТМ-6300-10/6 ЭКСГ. №8',
    'Y_ЭКСГАУСТЕР А/М №8_ПОДШИПНИК ОПОРНО-УПОРНЫЙ ЭКСГ. №8',
    'Y_ЭКСГАУСТЕР А/М №8_МАСЛОПРОВОДЫ ЭКСГАУСТЕРА №8',
    'Y_ЭКСГАУСТЕР А/М №4_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №4',
    'Y_ЭКСГАУСТЕР А/М №4_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №4',
    'Y_ЭКСГАУСТЕР А/М №4_УЛИТА ЭКСГ. №4',
    'Y_ЭКСГАУСТЕР А/М №4_РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №4',
    'Y_ЭКСГАУСТЕР А/М №4_МАСЛОНАСОС РАБОЧИЙ ЭКСГ. №4',
    'Y_ЭКСГАУСТЕР А/М №4_ЭЛЕКТРОАППАРАТУРА ЭКСГ. №4',
    'Y_ЭКСГАУСТЕР А/М №4_ПОДШИПНИК ОПОРНЫЙ №2 ЭКСГ. №4',
    'Y_ЭКСГАУСТЕР А/М №4_ЗАДВИЖКА ЭКСГ. №4',
    'Y_ЭКСГАУСТЕР А/М №4_ГСМ ЭКСГ. №4',
    'Y_ЭКСГАУСТЕР А/М №4_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№4 Т.1',
    'Y_ЭКСГАУСТЕР А/М №4_ТСМТ-101-010-50М-400 ТЕРМОПР.ПОДШ.Т.1',
    'Y_ЭКСГАУСТЕР А/М №4_РОТОР ЭКСГ. №4',
    'Y_ЭКСГАУСТЕР А/М №4_МАСЛООХЛАДИТЕЛЬ  М-05-1 ЭКСГ. №4',
    'Y_ЭКСГАУСТЕР А/М №4_ПОДШИПНИК ОПОРНО-УПОРНЫЙ ЭКСГ. №4',
    'Y_ЭКСГАУСТЕР А/М №4_ЭКСГАУСТЕР А/М №4',
    'Y_ЭКСГАУСТЕР А/М №4_ЗАП. И РЕГ. АРМАТУРА ЭКСГ.№4',
    'Y_ЭКСГАУСТЕР А/М №4_ПОДШИПНИК ОПОРНЫЙ №1 ЭКСГ. №4',
    'Y_ЭКСГАУСТЕР А/М №4_ЭЛ/ДВИГАТЕЛЬ ГАЗ. ЗАДВИЖКИ ЭКСГ. №4',
    'Y_ЭКСГАУСТЕР А/М №4_КЛ1 ТР№4 ДО ЭД ЭКСГАУСТЕРА №4',
    'Y_ЭКСГАУСТЕР А/М №4_КЛ2 ТР№4 ДО ЭД ЭКСГАУСТЕРА №4',
    'Y_ЭКСГАУСТЕР А/М №4_ТР-Р ТМ-4000-10/6 ЭКСГ. №4',
    'Y_ЭКСГАУСТЕР А/М №4_ТИРИСТ. ВОЗБУДИТЕЛЬ ВТ-РЭМ-400 ЭКСГ4 ВУ1',
    'Y_ЭКСГАУСТЕР А/М №4_МАСЛОПРОВОДЫ ЭКСГ №4',
    'Y_ЭКСГАУСТЕР А/М №7_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №7',
    'Y_ЭКСГАУСТЕР А/М №7_КОРПУС ЭКСГ. №7',
    'Y_ЭКСГАУСТЕР А/М №7_ПОДШИПНИК ОПОРНЫЙ №1',
    'Y_ЭКСГАУСТЕР А/М №7_ЗАДВИЖКА ЭКСГ. №7',
    'Y_ЭКСГАУСТЕР А/М №7_ПОДШИПНИК ОПОРНЫЙ №2',
    'Y_ЭКСГАУСТЕР А/М №7_РОТОР ЭКСГ. №7',
    'Y_ЭКСГАУСТЕР А/М №7_ПОДШИПНИК ОПОРНО-УПОРНЫЙ ЭКСГ. №7',
    'Y_ЭКСГАУСТЕР А/М №7_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№7 Т.3',
    'Y_ЭКСГАУСТЕР А/М №7_РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №7',
    'Y_ЭКСГАУСТЕР А/М №7_ТСМТ-101-010-50М-80 ТЕРМОПРЕОБР.МАСЛО',
    'Y_ЭКСГАУСТЕР А/М №7_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №7',
    'Y_ЭКСГАУСТЕР А/М №7_ГСМ ЭКСГ. №7',
    'Y_ЭКСГАУСТЕР А/М №7_КОЖУХ МУФТЫ ЭКСГ. №7',
    'Y_ЭКСГАУСТЕР А/М №7_ЭКСГАУСТЕР А/М №7',
    'Y_ЭКСГАУСТЕР А/М №7_МАСЛОПРОВОДЫ ЭКСГАУСТЕРА №7',
    'Y_ЭКСГАУСТЕР А/М №7_МАСЛООХЛАДИТЕЛЬ  М-05-1 ЭКСГ. №7',
    'Y_ЭКСГАУСТЕР А/М №7_САПФИР 22 МДД ПЕРЕПАД ДАВЛ. НА ЦИКЛОНЕ',
    'Y_ЭКСГАУСТЕР А/М №7_ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №7',
    'Y_ЭКСГАУСТЕР А/М №7_МАСЛОНАСОС РАБОЧИЙ ЭКСГ. №7',
    'Y_ЭКСГАУСТЕР А/М №7_МАСЛОНАСОС ШЕСТЕРЕНЧАТЫЙ (ПУСК.) ЭКСГ.№7',
    'Y_ЭКСГАУСТЕР А/М №7_МАСЛЯНЫЙ ФИЛЬТР ЭКСГАУСТЕРА №7',
    'Y_ЭКСГАУСТЕР А/М №7_ТР-Р ТМ-4000-10/6 ЭКСГ. №7',
    'Y_ЭКСГАУСТЕР А/М №7_ТСМТ-101-010-50М-200 ТЕРМОПР.ПОДШ.Т.3',
    'Y_ЭКСГАУСТЕР А/М №7_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№7 Т.2',
    'Y_ЭКСГАУСТЕР А/М №7_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№7 Т.4',
    'Y_ЭКСГАУСТЕР А/М №7_ТИРИСТОРНЫЙ ВОЗБУДИТЕЛЬ ТВ-400 ЭКСГ7 ВУ1',
    'Y_ЭКСГАУСТЕР А/М №7_ТСМТ-101-010-50М-400 ТЕРМОПР.ПОДШ.Т.2',
    'Y_ЭКСГАУСТЕР А/М №7_КЛ1 ТР№7 ДО ЭД ЭКСГАУСТЕРА №7',
    'Y_ЭКСГАУСТЕР А/М №7_МЕТРАН-100 ДАТЧИКИ ДАВЛЕНИЯ ЭКСГ.№7',
    'Y_ЭКСГАУСТЕР А/М №7_РЕГУЛИРУЮЩАЯ АППАРАТУРА ЭКСГАУСТЕРА №7',
    'Y_ЭКСГАУСТЕР А/М №7_ЭЛ/ДВИГАТЕЛЬ ГАЗ. ЗАДВИЖКИ ЭКСГ. №7',
    'Y_ЭКСГАУСТЕР А/М №7_ТСМТ-101-010-50М-400 ТЕРМОПР.ПОДШ.Т.1',
    'Y_ЭКСГАУСТЕР А/М №6_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №6',
    'Y_ЭКСГАУСТЕР А/М №6_КОРПУС ЭКСГ. №6',
    'Y_ЭКСГАУСТЕР А/М №6_ЗАДВИЖКА ЭКСГ. №6',
    'Y_ЭКСГАУСТЕР А/М №6_ПОДШИПНИК ОПОРНЫЙ №2',
    'Y_ЭКСГАУСТЕР А/М №6_РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №6',
    'Y_ЭКСГАУСТЕР А/М №6_ПОДШИПНИК ОПОРНО-УПОРНЫЙ ЭКСГ. №6',
    'Y_ЭКСГАУСТЕР А/М №6_САПФИР 22 МДД РАЗРЕЖЕНИЕ В КОЛЛЕКТОРЕ',
    'Y_ЭКСГАУСТЕР А/М №6_МАСЛОБАК ЭКСГ. №6',
    'Y_ЭКСГАУСТЕР А/М №6_ПОДШИПНИК ОПОРНЫЙ №1',
    'Y_ЭКСГАУСТЕР А/М №6_КОЖУХ МУФТЫ ЭКСГ. №6',
    'Y_ЭКСГАУСТЕР А/М №6_РОТОР ЭКСГ. №6',
    'Y_ЭКСГАУСТЕР А/М №6_ТР-Р ТМ-4000-10/6 ЭКСГ. №6',
    'Y_ЭКСГАУСТЕР А/М №6_ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГ №6',
    'Y_ЭКСГАУСТЕР А/М №6_МАСЛООХЛАДИТЕЛЬ  М-05-1 ЭКСГ. №6',
    'Y_ЭКСГАУСТЕР А/М №6_САПФИР 22 МДД ПЕРЕПАД ДАВЛ. НА ЦИКЛОНЕ',
    'Y_ЭКСГАУСТЕР А/М №6_МАСЛОНАСОС РАБОЧИЙ ЭКСГ. №6',
    'Y_ЭКСГАУСТЕР А/М №6_ГАЗОВАЯ ЗАДВИЖКА ЭКСГАУСТЕРА А/М №6',
    'Y_ЭКСГАУСТЕР А/М №6_ЭКСГАУСТЕР А/М №6',
    'Y_ЭКСГАУСТЕР А/М №6_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №6',
    'Y_ЭКСГАУСТЕР А/М №6_ТИРИСТОРНЫЙ ВОЗБУДИТЕЛЬ ТВ-400 ЭКСГ6 ВУ1',
    'Y_ЭКСГАУСТЕР А/М №6_ДВИГАТЕЛЬ ПУСКОВОГО МАСЛОНАСОСА ЭКСГ. №6',
    'Y_ЭКСГАУСТЕР А/М №6_МАСЛОНАСОС ШЕСТЕРЕНЧАТЫЙ (ПУСК.) ЭКСГ.№6',
    'Y_ЭКСГАУСТЕР А/М №6_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№6 Т.3',
    'Y_ЭКСГАУСТЕР А/М №6_МАСЛОПРОВОДЫ ЭКСГ №6',
    'Y_ЭКСГАУСТЕР А/М №6_КЛ1 ТР№6 ДО ЭД ЭКСГАУСТЕРА №6',
    'Y_ЭКСГАУСТЕР А/М №6_КЛ2 ТР№6 ДО ЭД ЭКСГАУСТЕРА №6',
    'Y_ЭКСГАУСТЕР А/М №6_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№6 Т.2',
]
len(Y_NAMES)

175

In [25]:
NAMES_TO_DEL_Y = [' ЭКСГАУСТЕРА', ' ЭКСГ.', ' №{}', '№{}']
EQUIP_NAMES_PREFIX_Y = [name.format(i) for name in NAMES_TO_DEL_Y for i in range(4, 10)]
EQUIP_NAMES_Y = ['Y_ЭКСГАУСТЕР А/М №'+ str(i) for i in range(4, 10)]

In [26]:
df_train_y_schema = pa.schema([(name, "int8") for name in Y_NAMES] + [(DATETIME_COL.upper(), pa.timestamp('s'))])

### Common info

In [27]:
df_train_y = pd.read_parquet(DATA_RAW_PATH / TRAIN_DATA_Y, schema=df_train_y_schema)
display(df_train_y.info(show_counts=True))
df_train_y.set_index(DATETIME_COL.upper(), drop=True, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9335034 entries, 0 to 9335033
Columns: 176 entries, Y_ЭКСГАУСТЕР А/М №9_ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №9 to DT
dtypes: datetime64[s](1), int8(175)
memory usage: 1.6 GB


None

### Separate info by eqiupment

In [28]:
df_train_y_dict = {}
df_train_y_cols_dict = {}

for equip_name_y, equip_name in zip(EQUIP_NAMES_Y, EQUIP_NAMES):
    df_train_y_dict[equip_name] = df_train_y[df_train_y.columns[df_train_y.columns.str.startswith(equip_name_y)]]
    df_train_y_dict[equip_name].columns = df_train_y_dict[equip_name].columns.str[len(EQUIP_NAMES_Y[0] + "_"):].str.replace('|'.join(EQUIP_NAMES_PREFIX_Y), '', regex=True).str.lower()
    df_train_y_dict[equip_name].index.name = df_train_y_dict[equip_name].index.name.lower()
    df_train_y_dict[equip_name] = df_train_y_dict[equip_name].loc[:, (df_train_y_dict[equip_name] == 0).all(axis=0)]
    df_train_y_cols_dict[equip_name] = df_train_y_dict[equip_name].columns.to_list()

Dataset timerange.

In [29]:
df_train_y_start_end = (df_train_y_dict[EQUIP_NAMES[0]].index.min(), df_train_y_dict[EQUIP_NAMES[0]].index.max())
df_train_y_shape = df_train_y_dict[EQUIP_NAMES[0]].shape
print("Date period: ", df_train_y_start_end)
print("Shape: ", df_train_y_shape)

Date period:  (Timestamp('2019-01-16 13:21:00'), Timestamp('2021-12-31 23:59:50'))
Shape:  (9335034, 1)


In [30]:
df_describe_info(df_train_y_dict[test_eqip])

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9335034 entries, 2019-01-16 13:21:00 to 2021-12-31 23:59:50
Data columns (total 1 columns):
 #   Column        Non-Null Count    Dtype
---  ------        --------------    -----
 0   маслопроводы  9335034 non-null  int8 
dtypes: int8(1)
memory usage: 80.1 MB


None

,маслопроводы
count,9335034.0
mean,0.0
std,0.0
min,0.0
0.2%,0.0
1%,0.0
5%,0.0
50%,0.0
95%,0.0
99%,0.0


NaNs portion: 0.000


In [31]:
for equip_name in EQUIP_NAMES:
    df_train_y_dict[equip_name] = df_train_y_dict[equip_name].groupby(pd.Grouper(freq=GROUP_FREQ)).aggregate(GROUP_FUNC_Y)

df_describe_info(df_train_y_dict[test_eqip])

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1555839 entries, 2019-01-16 13:21:00 to 2021-12-31 23:59:00
Freq: T
Data columns (total 1 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   маслопроводы  1555839 non-null  float64
dtypes: float64(1)
memory usage: 23.7 MB


None

,маслопроводы
count,1555839.0
mean,0.0
std,0.0
min,0.0
0.2%,0.0
1%,0.0
5%,0.0
50%,0.0
95%,0.0
99%,0.0


NaNs portion: 0.000


In [32]:
# for equip_name in EQUIP_NAMES:
#     df_train_y_dict[equip_name].to_csv(DATA_INTER_PATH / (equip_name + "_y" + SAVE_EXT))

## 2.3. Metainformation

In [33]:
EQUIP_NAMES_META = {equip_name_y[len('Y_'):]: equip_name for equip_name_y, equip_name in zip(EQUIP_NAMES_Y, EQUIP_NAMES)}
FAILURE_PART_COL = 'название_тех_места'
DESCRIPTION_COL = 'описание'
EQUIP_NAME = 'имя_машины'
FAILURE_TYPE_COL = 'вид_сообщения'
PLANNED_COL = 'текст_группы_кодов'


SATRT_END_FAILURE_DATES = ['дата_начала_неисправности', 'дата_устранения_неисправности']
PLANNED_CODES = {
    'технические плановые': 0, 
    'технические неплановые': 1,
    'технологические плановые': 0, 
    'технологические неплановые': 1, 
    'внешние организации': 1
}
FAILURES_CODES = {"shutdown": "M1", "malfunction": "M3"}

In [34]:
df_meta = pd.read_excel(DATA_RAW_PATH / METAINFO, index_col=None)
df_meta.columns = df_meta.columns.str.lower()
df_describe_info(df_meta)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981 entries, 0 to 980
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   unnamed: 0                     981 non-null    int64         
 1   машина                         981 non-null    object        
 2   имя_машины                     981 non-null    object        
 3   тех_место                      981 non-null    object        
 4   название_тех_места             981 non-null    object        
 5   вид_сообщения                  981 non-null    object        
 6   описание                       981 non-null    object        
 7   дата_начала_неисправности      981 non-null    datetime64[ns]
 8   дата_устранения_неисправности  851 non-null    datetime64[ns]
 9   текст_группы_кодов             83 non-null     object        
dtypes: datetime64[ns](2), int64(1), object(7)
memory usage: 76.8+ KB


None

,unnamed: 0,дата_начала_неисправности,дата_устранения_неисправности
count,981.000000,981,851
mean,1183.237513,2020-07-02 17:11:04.968399616,2020-07-28 03:07:27.599294976
min,390.000000,2019-01-21 00:00:00,2019-01-22 00:00:00
0.2%,391.960000,2019-01-21 15:27:51.680000,2019-01-26 06:28:06
1%,400.800000,2019-01-27 10:58:40.400000,2019-02-04 10:50:50
5%,459.000000,2019-02-25 13:38:04,2019-03-19 14:19:19
50%,1115.000000,2020-06-26 14:41:04,2020-07-27 00:00:00
95%,2103.000000,2021-11-09 23:46:00,2021-10-18 12:00:00
99%,2147.200000,2021-12-20 18:06:20.400000,2021-12-20 06:53:35
max,2159.000000,2021-12-29 16:27:02,2021-12-30 00:00:00


NaNs portion: 0.915


In [35]:
df_meta[FAILURE_PART_COL] = df_meta[FAILURE_PART_COL].str.replace('|'.join(EQUIP_NAMES_PREFIX_Y), '', regex=True).str.lower()
df_meta.drop("unnamed: 0", axis=1, inplace=True)
df_meta[EQUIP_NAME].replace(EQUIP_NAMES_META, inplace=True)

### Explore

In [36]:
# Explore specific equipment and dates
test_eqip = "эксгаустер 6"
data_filter = df_train_y_dict[test_eqip].index >= "2021-12-28 12:23:00"

print("Оборудование в ремонте:")
display(df_train_y_dict[test_eqip].loc[data_filter, (df_train_y_dict[test_eqip][data_filter] > 0).all(axis=0)])

Оборудование в ремонте:


""
dt
2021-12-28 12:23:00
2021-12-28 12:24:00
2021-12-28 12:25:00
2021-12-28 12:26:00
2021-12-28 12:27:00
...
2021-12-31 23:55:00
2021-12-31 23:56:00
2021-12-31 23:57:00


In [37]:
print("Ремонты без указания даты устранения неисправности:")
df_meta[df_meta["дата_устранения_неисправности"].isin([pd.NaT])]

Ремонты без указания даты устранения неисправности:


,машина,имя_машины,тех_место,название_тех_места,вид_сообщения,описание,дата_начала_неисправности,дата_устранения_неисправности,текст_группы_кодов
87,CH-AGP-AG2/009-005,эксгаустер 8,CH-AGP-AG2/009-005-002-004,маслоохладитель м-05-1,M3,греются подшипники,2019-04-08 21:50:44,NaT,NaN
162,CH-AGP-AG2/007-005,эксгаустер 7,CH-AGP-AG2/007-005-001-002,ротор,M3,течь масла,2019-06-18 18:31:12,NaT,NaN
187,CH-AGP-AG2/003-005,эксгаустер 5,CH-AGP-AG2/003-005-003-002,редуктор газ. задвижки,M3,нет смазки,2019-08-13 16:37:54,NaT,NaN
197,CH-AGP-AG2/011-005,эксгаустер 9,CH-AGP-AG2/011-005-002-001,маслонасос рабочий,M3,неисправность,2019-08-21 19:32:00,NaT,NaN
198,CH-AGP-AG2/011-005,эксгаустер 9,CH-AGP-AG2/011-005-001-003,подшипник опорно-упорный,M3,утечка масла,2019-08-21 19:40:37,NaT,NaN
...,...,...,...,...,...,...,...,...,...
972,AA2/002-006,эксгаустер 5,AA2/002-006-003-003,задвижка,M3,Деформация кожуха,2021-12-21 13:32:00,NaT,NaN
973,AA2/001-006,эксгаустер 4,AA2/001-006-003-003,задвижка,M3,Неисправны датчики.,2021-12-21 14:34:35,NaT,NaN
977,AA2/005-006,эксгаустер 8,AA2/005-006-003-002,редуктор газ. задвижки,M3,Смазка редуктора.,2021-12-28 09:23:00,NaT,NaN
978,AA2/003-006,эксгаустер 6,AA2/003-006-001-001,корпус,M3,Прососы,2021-12-28 12:23:00,NaT,NaN


In [38]:
print("Уникальных описаний поломок среди всех поломок:", f"{df_meta[DESCRIPTION_COL].nunique()}/{len(df_meta)}")

Уникальных описаний поломок среди всех поломок: 555/981


In [39]:
df_meta[PLANNED_COL] = df_meta[PLANNED_COL].str.lower().replace(PLANNED_CODES)

In [40]:
# df_meta.to_csv(DATA_INTER_PATH / ("metainfo" + SAVE_EXT))

# 3. EDA

In [41]:
df_meta[(df_meta[EQUIP_NAME] == test_eqip) & (df_meta[FAILURE_TYPE_COL] == FAILURES_CODES['malfunction'])].head(5)

,машина,имя_машины,тех_место,название_тех_места,вид_сообщения,описание,дата_начала_неисправности,дата_устранения_неисправности,текст_группы_кодов
15,CH-AGP-AG2/005-005,эксгаустер 6,CH-AGP-AG2/005-005-001-005,электродвигатель дспу-140-84-4,M3,"износ подшипников, обмотки статора",2019-01-30 05:30:12,2019-01-30 16:30:50,NaN
25,CH-AGP-AG2/005-005,эксгаустер 6,CH-AGP-AG2/005-005-001-001,корпус,M3,износ уплотнения разъема улиты,2019-02-07 10:00:35,2019-03-19 00:00:00,NaN
44,CH-AGP-AG2/005-005,эксгаустер 6,CH-AGP-AG2/005-005-003-003,задвижка,M3,клинит задвижку при закрытии,2019-02-13 19:05:46,2019-03-11 00:00:00,NaN
45,AA2/003-006,эксгаустер 6,AA2/003-006-001-012-002,подшипник опорный №2,M3,течь масла,2019-02-25 00:00:00,2019-03-06 00:00:00,NaN
46,CH-AGP-AG2/005-005,эксгаустер 6,CH-AGP-AG2/005-005-003-003,задвижка,M3,нет колпака,2019-02-25 13:02:18,2019-07-01 00:00:00,NaN


In [42]:
df_meta[(df_meta[EQUIP_NAME] == test_eqip) & (df_meta[FAILURE_TYPE_COL] == FAILURES_CODES['shutdown'])]

,машина,имя_машины,тех_место,название_тех_места,вид_сообщения,описание,дата_начала_неисправности,дата_устранения_неисправности,текст_группы_кодов
121,CH-AGP-AG2/005-005,эксгаустер 6,CH-AGP-AG2/005-005-003-003,задвижка,M1,упала задвижка на эксгаустере,2019-05-01 16:29:38,2019-05-01 19:18:04,1.0
123,AA2/003-006,эксгаустер 6,AA2/003-006-003-003,задвижка,M1,упала задвижка на эксгаустере,2019-05-01 19:18:04,2019-05-01 19:18:04,1.0
344,CH-AGP-AG2/005-005,эксгаустер 6,CH-AGP-AG2/005-005-003-003,задвижка,M1,остановка эксгаустера №6 (обрыв гайки),2020-01-16 06:18:04,2020-01-16 08:12:18,1.0
350,CH-AGP-AG2/005-005,эксгаустер 6,CH-AGP-AG2/005-005-003-003,задвижка,M1,остановка эксгаустера №6 (монтаж гайки),2020-01-17 10:09:38,2020-01-17 13:19:04,1.0
622,CH-AGP-AG2/005-005,эксгаустер 6,CH-AGP-AG2/005-005-003,газовая задвижка а/м,M1,выпуск аглопыли из газового коллектора,2020-12-02 17:10:12,2020-12-02 17:27:14,0.0
738,CH-AGP-AG2/005-005,эксгаустер 6,CH-AGP-AG2/005-005-003-003,задвижка,M1,остановка эк-ра №6 (обрыв гайки),2021-04-16 13:23:07,2021-04-16 18:40:46,1.0
740,AA2/003-006,эксгаустер 6,AA2/003-006,эксгаустер а/м,M1,Согласованное ТО,2021-04-17 06:06:56,2021-04-17 06:27:58,0.0
741,CH-AGP-AG2/005-005,эксгаустер 6,CH-AGP-AG2/005-005-003-003,задвижка,M1,остановка эк-ра №6 (обрыв штока),2021-04-18 18:58:57,2021-04-18 23:22:29,1.0


In [43]:
df_train_dict[test_eqip].columns

Index(['ток ротора 1', 'ток ротора 2', 'ток статора',
       'давление масла в системе', 'температура подшипника на опоре 1',
       'температура подшипника на опоре 2',
       'температура подшипника на опоре 3',
       'температура подшипника на опоре 4', 'температура масла в системе',
       'температура масла в маслоблоке', 'вибрация на опоре 1',
       'вибрация на опоре 2', 'вибрация на опоре 3', 'вибрация на опоре 4',
       'вибрация на опоре 3. продольная.', 'вибрация на опоре 4. продольная.'],
      dtype='object')

In [87]:
current_stator_cols = ['ток ротора 1', 'ток ротора 2', 'ток статора']
pressure_oil = ['давление масла в системе']
bearing_temp = ['температура подшипника на опоре 1', 'температура подшипника на опоре 2', 'температура подшипника на опоре 3', 'температура подшипника на опоре 4']
oil_temp = ['температура масла в системе', 'температура масла в маслоблоке']
vibration_x = ['вибрация на опоре 1', 'вибрация на опоре 2', 'вибрация на опоре 3', 'вибрация на опоре 4']
vibration_y = ['вибрация на опоре 3. продольная.', 'вибрация на опоре 4. продольная.']

visual_eqip = "эксгаустер 6" # test_eqip
visula_dates = ('2021-03-15 00:00:00', '2021-04-30 00:00:00') # ('2021-04-01 00:00:00', '2021-04-30 00:00:00')

plot_position = [0.05, 0.95]
y_plots = [current_stator_cols, pressure_oil, bearing_temp + oil_temp, vibration_x + vibration_y]
y_axis_names = ['current_v', 'pressure_pa', 'temp_c', 'vib_µm']
y_axis_map_names = [f"y{i}" for i in range(1, len(y_plots)+1)]
y_places = [0, 0.05, 0.95, 1.00]
colors = plt.get_cmap('rainbow', 20)
smooth_window = 10

In [88]:
# initialize a Figure object and store it in
# a variable fig
fig = go.Figure()
 
# add x and y values for the 1st scatter
# plot and name the yaxis as yaxis1 values

df_to_visual = df_train_dict[visual_eqip].copy()
df_to_visual = df_to_visual[(df_to_visual.index >= visula_dates[0]) & (df_to_visual.index <= visula_dates[1])]
if smooth_window > 0:
    df_to_visual = df_to_visual.rolling(smooth_window).mean()

for y_axis_name, cols in zip(y_axis_map_names, y_plots):
    for col in cols:
        fig.add_trace(go.Scatter(
            x=df_to_visual[col].dropna().index,
            y=df_to_visual[col].dropna(),
            name=col,
            yaxis=y_axis_name,
        ))

yaxis_dict = {}
for i, (y_axis_name, y_pos) in enumerate(zip(y_axis_names, y_places)):
    color = to_hex(colors(i), keep_alpha=False)
    name = f'yaxis{i+1}'
    add_params = dict(
        anchor="free",  # specifying x - axis has to be the fixed
        overlaying="y",  # specifyinfg y - axis has to be separated
    )
    if i == 0:
        name = "yaxis"
        add_params = {}
    yaxis_dict[name] = dict(
        title=y_axis_name,
        titlefont=dict(color=color),
        tickfont=dict(color=color),
        side="left" if y_pos <= plot_position[0] else "right",
        position=y_pos,
        **add_params,
    )

# Create axis objects
fig.update_layout(
    # split the x-axis to fraction of plots in
    # proportions
    xaxis=dict(
        domain=plot_position
    ),
    # pass the y-axis title, titlefont, color
    # and tickfont as a dictionary and store
    # it an variable yaxis
    **yaxis_dict
)

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=7, label="7d", step="day", stepmode="backward"),
            dict(count=14, label="14d", step="day", stepmode="todate"),
            dict(step="all")
        ])
    )
)

# Update layout of the plot namely title_text, width
# and place it in the center using title_x parameter
# as shown
fig.update_layout(
    title_text=f"Анализ работы агрегата {test_eqip} за {visula_dates[0]}-{visula_dates[1]}",
    xaxis_title=DATETIME_COL,
    hovermode='x unified',
    height=1000,
    width=1600,
    title_x=0.5
)
# fig.update_traces(mode="markers+lines", hovertemplate=None)
# fig.update_layout(hovermode="x")

fig.write_html(PIC_EXPLORATORY_PATH / f"{visual_eqip}_{visula_dates[0].replace(':', '-')}_{visula_dates[1].replace(':', '-')}.html")

In [89]:
df_equip = df_meta[df_meta[EQUIP_NAME] == visual_eqip]
df_equip[(df_equip[SATRT_END_FAILURE_DATES[0]] >= visula_dates[0]) & (df_equip[SATRT_END_FAILURE_DATES[1]] <= visula_dates[1])]

,машина,имя_машины,тех_место,название_тех_места,вид_сообщения,описание,дата_начала_неисправности,дата_устранения_неисправности,текст_группы_кодов
738,CH-AGP-AG2/005-005,эксгаустер 6,CH-AGP-AG2/005-005-003-003,задвижка,M1,остановка эк-ра №6 (обрыв гайки),2021-04-16 13:23:07,2021-04-16 18:40:46,1.0
740,AA2/003-006,эксгаустер 6,AA2/003-006,эксгаустер а/м,M1,Согласованное ТО,2021-04-17 06:06:56,2021-04-17 06:27:58,0.0
741,CH-AGP-AG2/005-005,эксгаустер 6,CH-AGP-AG2/005-005-003-003,задвижка,M1,остановка эк-ра №6 (обрыв штока),2021-04-18 18:58:57,2021-04-18 23:22:29,1.0
742,AA2/003-006,эксгаустер 6,AA2/003-006-003-003,задвижка,M3,Обрыв гайки задвижки,2021-04-19 00:55:00,2021-04-19 04:07:28,NaN
748,AA2/003-006,эксгаустер 6,AA2/003-006-003-002,редуктор газ. задвижки,M3,Отсутствие гайки,2021-04-20 07:43:24,2021-04-28 12:15:56,NaN


In [100]:
# df_equip[(df_equip[SATRT_END_FAILURE_DATES[1]] >= '') & (df_equip[SATRT_END_FAILURE_DATES[1]] <= '2019-05-01 16:29:38')]

# 4. Feature engineering